In [ ]:
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
from PIL import Image
from IPython.display import display
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.nn.functional as F
from transformers import TextStreamer

In [ ]:
# Hyperparameters
SECONDS_BEFORE = 0.5  # number of seconds before the collision for image frame.

In [ ]:
PROJECT_DIR = "nexcar-challenge"
DATA_DIR = "data"

train_csv_path = f"/content/drive/MyDrive/{PROJECT_DIR}/{DATA_DIR}/train.csv"
train_videos_folder = f"/content/drive/MyDrive/{PROJECT_DIR}/{DATA_DIR}/train/"

df = pd.read_csv(train_csv_path)
# df = df[df['target']==1]
df.head()

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, data, videos_folder, transform=None):
        self.data = data
        self.videos_folder = videos_folder
        self.transform = transform  # Any image transformations (e.g., augmentations)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]

        vid_path = os.path.join(
            self.videos_folder, f"{str(int(row['id'])).zfill(5)}.mp4"
        )

        time = (
            0.0
            if np.isnan(row["time_of_event"])
            else row["time_of_event"] - SECONDS_BEFORE
        )
        image = self.get_frame(vid_path, time)

        label = row["target"]

        # Apply transformations
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label), row['id']  # Convert label to tensor

    def get_frame(self, video_path, time_sec):
        cap = cv2.VideoCapture(video_path)
        cap.set(cv2.CAP_PROP_POS_MSEC, time_sec * 1000)

        success, frame = cap.read()
        cap.release()

        if not success:
            raise ValueError(f"Failed to read frame at {time_sec} seconds.")

        # Convert BGR to RGB
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        return Image.fromarray(frame)


transform = transforms.Compose(
    [
        transforms.Resize((480, 854)),  # Resize to a standard size
    ]
)

dataset = ImageDataset(df, train_videos_folder, transform=transform)
dataloader = DataLoader(dataset, shuffle=True)

In [ ]:
image, label, _ = dataset[0]
display(image)
print(label)

In [ ]:
instruction = """Examine the provided dashcam image and analyze the positions and trajectories of all visible vehicles and obstacles. Based solely on the visual cues in the image, determine if there is an imminent risk of collision. Answer with only "Yes" or "No"."""


def convert_to_conversation(image, label):
    conversation = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": instruction},
                {"type": "image", "image": image},
            ],
        },
        {
            "role": "assistant",
            "content": [{"type": "text", "text": "Yes" if label else "No"}],
        },
    ]
    return {"messages": conversation}


converted_dataset = [
    convert_to_conversation(image, label) for image, label, _ in tqdm(dataset)
]

In [ ]:
converted_dataset[0]

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab and Kaggle notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
    !pip install --no-deps cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [ ]:
from unsloth import FastVisionModel  # FastLanguageModel for LLMs
import torch

model, tokenizer = FastVisionModel.from_pretrained(
    f"/content/drive/MyDrive/{PROJECT_DIR}/pretrained/finetuned_llama/",
    load_in_4bit=True,  # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for long context
)

In [ ]:
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers=True,  # False if not finetuning vision layers
    finetune_language_layers=True,  # False if not finetuning language layers
    finetune_attention_modules=True,  # False if not finetuning attention layers
    finetune_mlp_modules=True,  # False if not finetuning MLP layers
    r=16,  # The larger, the higher the accuracy, but might overfit
    lora_alpha=16,  # Recommended alpha == r at least
    lora_dropout=0,
    bias="none",
    random_state=3407,
    use_rslora=False,  # We support rank stabilized LoRA
    loftq_config=None,  # And LoftQ
    # target_modules = "all-linear", # Optional now! Can specify a list if needed
)

In [ ]:
FastVisionModel.for_inference(model)  # Enable for inference!

image, label, _ = dataset[0]
conversation = convert_to_conversation(image, label)

input_text = tokenizer.apply_chat_template(
    conversation["messages"], add_generation_prompt=True
)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer, skip_prompt=True)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=128,
    use_cache=True,
    temperature=1.5,
    min_p=0.1
)

In [ ]:
FastVisionModel.for_inference(model)  # Enable for inference!

image, label, _ = dataset[0]
conversation = convert_to_conversation(image, label)

input_text = tokenizer.apply_chat_template(
    conversation["messages"], add_generation_prompt=True
)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer, skip_prompt=True)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=128,
    use_cache=True,
    temperature=1.5,
    min_p=0.1
)

In [ ]:
correct = 0
total = len(dataset)

for i, (image, target, _) in enumerate(tqdm(dataset)):
    conversation = convert_to_conversation(image, label)
    input_text = tokenizer.apply_chat_template(
        conversation["messages"], add_generation_prompt=True
    )
    inputs = tokenizer(
        image,
        input_text,
        add_special_tokens=False,
        return_tensors="pt",
    ).to("cuda")

    # Generate output and decode text
    output_tokens = model.generate(
        **inputs, max_new_tokens=128, use_cache=True, temperature=1.5, min_p=0.1
    )
    output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True).lower()

    # Evaluate the output:
    if (target == 1 and "yes" in output_text.lower()) or (
        target != 1 and "no" in output_text.lower()
    ):
        correct += 1

accuracy = correct / total
print("Accuracy:", accuracy)

In [ ]:
test_csv_path = f"/content/drive/MyDrive/{PROJECT_DIR}/{DATA_DIR}/test.csv"
test_videos_folder = f"/content/drive/MyDrive/{PROJECT_DIR}/{DATA_DIR}/test/"
test_set = ImageDataset(df, train_videos_folder, transform=transform)

In [ ]:
results = []

for image, target, id in test_set:
    conversation = convert_to_conversation(image, label)
    input_text = tokenizer.apply_chat_template(
        conversation["messages"], add_generation_prompt=True
    )

    # Tokenize the image and text inputs and move them to CUDA.
    inputs = tokenizer(
        image,
        input_text,
        add_special_tokens=False,
        return_tensors="pt",
    ).to("cuda")

    output_tokens = model.generate(
        **inputs, max_new_tokens=128, use_cache=True, temperature=1.5, min_p=0.1
    )

    target = 1 if "yes" in output_text.lower() else 0
    results.append({"id": id, "target": target})

df = pd.DataFrame(results)
df.to_csv("test_results.csv", index=False)
print("Test set results saved to 'test_results.csv'")